##### Copyright 2018 Los autores de TensorFlow. [Con licencia de Apache License, versión 2.0](#scrollTo=bPJq2qP2KE3u) .

In [ ]:
// #@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
// Licensed under the Apache License, Version 2.0 (the "License");
// you may not use this file except in compliance with the License.
// You may obtain a copy of the License at
//
// https://www.apache.org/licenses/LICENSE-2.0
//
// Unless required by applicable law or agreed to in writing, software
// distributed under the License is distributed on an "AS IS" BASIS,
// WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
// See the License for the specific language governing permissions and
// limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/swift/tutorials/raw_tensorflow_operators"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/swift/blob/master/docs/site/tutorials/raw_tensorflow_operators.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/swift/blob/master/docs/site/tutorials/raw_tensorflow_operators.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a></td>
</table>

# Operadores de TensorFlow sin procesar

Swift para TensorFlow, basado en TensorFlow, adopta un enfoque nuevo para el diseño de API. Las API se seleccionan cuidadosamente a partir de bibliotecas establecidas y se combinan con nuevas expresiones idiomáticas. Esto significa que no todas las API de TensorFlow estarán disponibles directamente como API de Swift, y nuestra selección de API necesita tiempo y esfuerzo dedicado para evolucionar. Sin embargo, no se preocupe si su operador de TensorFlow favorito no está disponible en Swift: la biblioteca de TensorFlow Swift le brinda acceso transparente a la mayoría de los operadores de TensorFlow, en el espacio de nombres `_Raw` .


Importa `TensorFlow` para comenzar.

In [ ]:
import TensorFlow

## Llamar a operadores sin procesar

Simplemente busque la función que necesita en el espacio de nombres `_Raw` mediante la finalización del código.

In [ ]:
_Raw.mul(Tensor([2.0, 3.0]), Tensor([5.0, 6.0]))

## Definición de un nuevo operador de multiplicación

Multiply ya está disponible como operador `*` en `Tensor` , pero pretendamos que queremos que esté disponible con un nuevo nombre como `.*` . Swift le permite agregar métodos o propiedades calculadas de forma retroactiva a tipos existentes mediante declaraciones de `extension` .

Ahora, agreguemos `.*` `Tensor` declarando una extensión y haga que esté disponible cuando el tipo `Scalar` del tensor se ajuste a [`Numeric`](https://developer.apple.com/documentation/swift/numeric) .

In [ ]:
infix operator .* : MultiplicationPrecedence

extension Tensor where Scalar: Numeric {
    static func .* (_ lhs: Tensor, _ rhs: Tensor) -> Tensor {
        return _Raw.mul(lhs, rhs)
    }
}

let x: Tensor<Double> = [[1.0, 2.0], [3.0, 4.0]]
let y: Tensor<Double> = [[8.0, 7.0], [6.0, 5.0]]
x .* y

## Definición de una derivada de una función envuelta

No solo puede definir fácilmente una API Swift para un operador de TensorFlow sin procesar, sino que también puede hacer que sea diferenciable para trabajar con la diferenciación automática de primera clase de Swift.

Para hacer `.*` Diferenciable, use el atributo `@derivative` en la función derivada y especifique la función original como un argumento de atributo bajo la etiqueta `of:` . Dado que el operador `.*` Se define cuando el tipo genérico `Scalar` ajusta a `Numeric` , no es suficiente para hacer que `Tensor<Scalar>` ajuste al protocolo `Differentiable` . Nacido con la seguridad de tipos, Swift nos recordará que agreguemos una restricción genérica en el atributo `@differentiable` para requerir que `Scalar` ajuste al protocolo `TensorFlowFloatingPoint` , lo que haría que `Tensor<Scalar>` ajuste a `Differentiable` .

```swift
@differentiable(where Scalar: TensorFlowFloatingPoint)
```

In [ ]:
infix operator .* : MultiplicationPrecedence

extension Tensor where Scalar: Numeric {
    @differentiable(where Scalar: TensorFlowFloatingPoint)
    static func .* (_ lhs: Tensor,  _ rhs: Tensor) -> Tensor {
        return _Raw.mul(lhs, rhs)
    }
}

extension Tensor where Scalar : TensorFlowFloatingPoint { 
    @derivative(of: .*)
    static func multiplyDerivative(
        _ lhs: Tensor, _ rhs: Tensor
    ) -> (value: Tensor, pullback: (Tensor) -> (Tensor, Tensor)) {
        return (lhs * rhs, { v in
            ((rhs * v).unbroadcasted(to: lhs.shape),
            (lhs * v).unbroadcasted(to: rhs.shape))
        })
    }
}

// Now, we can take the derivative of a function that calls `.*` that we just defined.
gradient(at: x, y) { x, y in
    (x .* y).sum()
}

## Más ejemplos

In [ ]:
let matrix = Tensor<Float>([[1, 2], [3, 4]])

print(_Raw.matMul(matrix, matrix, transposeA: true, transposeB: true))
print(_Raw.matMul(matrix, matrix, transposeA: true, transposeB: false))
print(_Raw.matMul(matrix, matrix, transposeA: false, transposeB: true))
print(_Raw.matMul(matrix, matrix, transposeA: false, transposeB: false))